In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime
import requests
import time
import os
import json

In [ ]:
gitcoin_resp = requests.get('https://gitcoin.co/grants/grants.json')
gitcoin_data = gitcoin_resp.json()

In [ ]:
gitcoin_data[:3]

[['Yew Assets', '0xb8c7fCA90863D00E033F749B81aB816608F9EF37'],
 ['Axiomatic Theory of Formulas',
  '0x36A0356d43EE4168ED24EFA1CAe3198708667ac0'],
 ['rocketh, a smart contract tool focusing on test and deployment.',
  '0xF78cD306b23031dE9E739A5BcDE61764e82AD5eF']]

In [ ]:
address_to_project = {str(y).lower():x for x,y in gitcoin_data}

In [ ]:
with open('grants.json', 'w') as f:
    json.dump(address_to_project, f)

In [ ]:
with open('nice_rounds.json', 'r') as f:
    nice_rounds = json.load(f)

In [ ]:
key = os.environ['ETHER_KEY']

In [ ]:
def get_transactions(address, page=1, offset=10000, startblock=0, endblock=99999999, erc20=False):
    """
    Docs: https://docs.etherscan.io/api-endpoints/accounts#get-a-list-of-normal-transactions-by-address
    Get `txlist` for an address

    args:
        address
    """
    endpoint = 'https://api.etherscan.io/api'
    action = 'txlist'
    if erc20:
        action = 'tokentx'
    params={
        'module':'account', 'action': action,
        'address': address, 'apikey': key,
        'startblock': startblock, 'endblock': endblock, 
        'page': page, 'offset': offset,
        'sort': 'asc', 
        }

    resp = requests.get(endpoint, params=params)
    try:
        data = resp.json()
        return data['result']
    except:
        print(resp)
        return resp

In [ ]:
test = get_transactions('0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae')
#test[:2]

In [ ]:
test2 = get_transactions('0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae', erc20=True)
#test2[:2]

In [ ]:
print(f'# gitcoin grants: {len(gitcoin_data)}')

# gitcoin grants: 2317


## Scrape by Contributor Address

* https://ethereum.stackexchange.com/questions/62007/how-to-get-the-blocks-between-timestamp

In [ ]:
round_rows = {}
round_logs = {}

In [ ]:
import time
t1 = time.time()

for k, d in nice_rounds.items():
    if int(k) < 9:
        continue
    print(k)
    round_rows[k] = []
    round_logs[k] = {
        'eth_more_than_10k': [],
        'erc20_more_than_10k': [],
        'errors': [],
    }

    print(d['startdate'], d['startblock'], d['enddate'], d['endblock'])


    for i, address in enumerate(d['addresses']):
        eth_rows = get_transactions(address, erc20=False, startblock=d['startblock'], endblock=d['endblock'])
        erc20_rows = get_transactions(address, erc20=True, startblock=d['startblock'], endblock=d['endblock'])
        
        round_rows[k] += eth_rows
        
        round_rows[k] += erc20_rows
        time.sleep(1)
        
        if len(eth_rows) >= 10000:
            round_logs[k]['eth_more_than_10k'].append(address)
            print('eth', address)
        if len(erc20_rows) >= 10000:
            round_logs[k]['erc20_more_than_10k'].append(address)
            print('erc20', address)
        if i % 200 == 0:
            print(i, time.time()-t1)
    successful_rows = [x for x in round_rows[k] if type(x) == dict]
    round_logs[k]['errors'] = [x for x in round_rows[k] if type(x) != dict]
    
    df = pd.DataFrame(successful_rows)
    df.to_csv(f'{k}_transactions.csv')
    
    print(k, len(df))

9
2021-03-10 00:00:00 12007407 2021-03-25 00:00:00 12104732
0 1.53371000289917
200 302.47893023490906
400 599.5939509868622
600 899.1229176521301
800 1202.637776851654
1000 1527.8158123493195
1200 1864.8179121017456
1400 2282.2234184741974
1600 2659.4628336429596
1800 3027.34783744812
2000 3333.512151002884
2200 3634.080769777298
2400 3945.56418466568
2600 4246.214415550232
2800 4551.711427688599
3000 4876.944789171219
3200 5178.573101520538
3400 5477.424495220184
3600 5793.987558841705
3800 6163.224286556244
4000 6504.22168302536
4200 6848.724138259888
4400 7152.528051137924
4600 7443.969089746475
4800 7746.802674770355
5000 8050.509400367737
5200 8352.333802700043
5400 8648.488475322723
5600 8950.102833747864
5800 9249.892159223557
6000 9553.173498153687
6200 9886.549406051636
6400 10260.022836208344
6600 10681.928089141846
6800 10987.012693881989
7000 11285.350445270538
7200 11573.024257421494
7400 11861.897775411606
7600 12153.88305425644
7800 12441.45426273346
8000 12730.389853954

In [ ]:
`

SyntaxError: invalid syntax (<ipython-input-14-f6460d7d9d2b>, line 1)

In [ ]:
round_rows[3]

,blockNumber,blockHash,timeStamp,hash,nonce,transactionIndex,from,to,value,gas,...,input,contractAddress,cumulativeGasUsed,txreceipt_status,gasUsed,confirmations,isError,tokenName,tokenSymbol,tokenDecimal
0,7446563,0x9c21410be34e5fed0235fc74ffd3bc38fc974ba94800...,1553635469,0xa5f11f4d68382ea88c42e3e2b14df78956d6105b3d2a...,429,44,0xe4b420f15d6d878dcd0df7120ac0fc1509ee9cab,0x00de4b13153673bcae2616b67bf822500d325fc3,90000000000000000,318730,...,0x,,6381981,1,21000,6132572,0,NaN,NaN,NaN
1,7451102,0xce528fcdcb48bc9eb8aa91fc89df784a41e422a05407...,1553696500,0x044b5e6cc119bd44375873825ef2317fd15c485f93d9...,1142,59,0x00de4b13153673bcae2616b67bf822500d325fc3,0x970cc8c95c614a7335c1487eb2e16848983d5fe6,100000000000000000,21000,...,0x,,2142129,1,21000,6128033,0,NaN,NaN,NaN
2,7451257,0x4a2815badc4adfb19978bb64fa8da086c5b298e09238...,1553698657,0xbe64fa2bf53a3bc980a9abb27c37a8302fd6b4540429...,1143,71,0x00de4b13153673bcae2616b67bf822500d325fc3,0x74373e74a4f9f8ee73b9964d62024dd6c95a7e8a,1775000000000000,21000,...,0x,,3835610,1,21000,6127878,0,NaN,NaN,NaN
3,7453063,0xfebad9eaf26c21dd9f6a0b79ae1aa069fb8667fb9968...,1553722157,0xacb7461e4fecc7a98a6061a72337c31909fddbfbef4f...,1144,63,0x00de4b13153673bcae2616b67bf822500d325fc3,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,0,68977,...,0x095ea7b3000000000000000000000000cf071db78d96...,,5108799,1,45985,6126072,0,NaN,NaN,NaN
4,7453078,0x09deadd2a495fda9ecde4f47ff22b4a45b007468a5d0...,1553722396,0x4f0f7c947a8caeffb0dd5ed31570ec8dc2afafc89d93...,1145,24,0x00de4b13153673bcae2616b67bf822500d325fc3,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,0,56043,...,0xa9059cbb00000000000000000000000068085e7f88e9...,,2582872,1,37362,6126057,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,7464442,0x1b17f3d54d553355566f6009917cc83e123fb390a053...,1553876626,0xd2df350cf6fb01ded94f53070a6a9cca6cb57b47e784...,9,112,0xfaca150c08ce2c8d704e10dbef1ba0e1d292feb0,0x448a5065aebb8e423f0896e6c5d525c040f59af3,200000000000000000,74349,...,deprecated,0xf53ad2c6851052a81b42133467480961b2321c09,6362293,NaN,34566,6114700,NaN,Pooled Ether,PETH,18
2093,7464481,0xbea64143a1fdf443755b51d361fe22526cdade7c914b...,1553877264,0x8292db8b13957d607274b7944e71807fc75258ad05fc...,10,65,0xfaca150c08ce2c8d704e10dbef1ba0e1d292feb0,0xfaca150c08ce2c8d704e10dbef1ba0e1d292feb0,7000000000000000000,78447,...,deprecated,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,3854959,NaN,37298,6114661,NaN,Sai Stablecoin v1.0,SAI,18
2094,7464512,0x707a3bcffca0418158c98dff947bd31694f1cbe962b0...,1553877642,0x42fb92ac3689d556322a6980566fe0b1e6c3bf131e01...,11,33,0xfaca150c08ce2c8d704e10dbef1ba0e1d292feb0,0xdb7fc5e20fe4567ececba1df82a234781aa37f8c,5000000000000000000,55947,...,deprecated,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,1982732,NaN,37298,6114630,NaN,Sai Stablecoin v1.0,SAI,18
2095,7502479,0x6582d62ab625c7b0fea77c725e1b5337519e8f5d9588...,1554391497,0x9bcdad7b3c0e410a8739422a21ef00b2edbd625b682d...,1436,166,0xfaca150c08ce2c8d704e10dbef1ba0e1d292feb0,0xdb7fc5e20fe4567ececba1df82a234781aa37f8c,1000000000000000000,204066,...,deprecated,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,6926902,NaN,119619,6076663,NaN,Sai Stablecoin v1.0,SAI,18


## Scrape by Grant Address

In [ ]:
SCRAPE_BY_GRANT_ADDRESS = False

In [ ]:
if SCRAPE_BY_GRANT_ADDRESS:
    t1 = time.time()

    total_rows = []
    name_to_address = {}
    investigate = []
    for i, (name, address) in enumerate(gitcoin_data):
        name_to_address[name] = address
        rows = get_txlist(address)
        if len(rows) >= 10000:
            investigate.append((name, address))
            print(name, address, len(rows))
        total_rows += rows
        time.sleep(1)
        if i % 100 == 0:
            print(i, time.time()-t1)

NameError: name 'SCRAPE_BY_GRANT_ADDRESS' is not defined

In [ ]:
# special_rows = []
# page = 1
# while page < 100:
#     page += 1
#     rows = my_get('0xB391AE8Fc7483770930d1dEAb5a898206E9cDb3F')
#     print(page, len(rows))
#     special_rows += rows
#     time.sleep(1)


In [ ]:
total_rows

[{'blockNumber': '9741106',
  'timeStamp': '1585147772',
  'hash': '0xaef1b028055d57d27ef3c1c27b52e0081449aaf5b6040cb0878f83d1507567e3',
  'nonce': '2960',
  'blockHash': '0x4ec40e185edfded6adff507653a0bb03daebb62214069fe220028a94236bcb5e',
  'transactionIndex': '65',
  'from': '0x00de4b13153673bcae2616b67bf822500d325fc3',
  'to': '0x01f1e9319982ab828203c0276727cf48065434d3',
  'value': '2500000000000000',
  'gas': '21000',
  'gasPrice': '10000000000',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0x',
  'contractAddress': '',
  'cumulativeGasUsed': '4420950',
  'gasUsed': '21000',
  'confirmations': '3543842'},
 {'blockNumber': '10395981',
  'timeStamp': '1593906601',
  'hash': '0x8645d41dfb512084a5f6c60b49caf541086a15cb46a08ba5538a95dd9303e9ba',
  'nonce': '1313684',
  'blockHash': '0x29d4467065cceb46430e358c1f658684845a8d80436f8bff504ce23ec3c21266',
  'transactionIndex': '40',
  'from': '0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0',
  'to': '0x9b66bcefa1becfe4b988e23cb1017

[{'blockNumber': '13198991',
  'timeStamp': '1631289899',
  'hash': '0x19447f4db56074a58ed898d82792b1840cae17cc9f4940a4e0403254682953dd',
  'nonce': '7',
  'blockHash': '0xad8c19b380dbf724d49032badc36a52f85adc3b498d176b1cec211a80fbec2bc',
  'transactionIndex': '21',
  'from': '0x0cbbc6ae48d9d3e14079aa15fab79a93b9f14796',
  'to': '0x447997d1dc7888a2ffc15826e7595a6f7dcc076b',
  'value': '40000000000000000',
  'gas': '21000',
  'gasPrice': '201000000000',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0x',
  'contractAddress': '',
  'cumulativeGasUsed': '1712459',
  'gasUsed': '21000',
  'confirmations': '303446'},
 {'blockNumber': '13199002',
  'timeStamp': '1631290114',
  'hash': '0x9069dd7c39248b635b4d4129a9f747b0fbed1df80b7e3cc76a7a75567017fdfd',
  'nonce': '0',
  'blockHash': '0xceb3d9ed463a983d7472fa197aabebefca533e950aab2553feae8a9017785147',
  'transactionIndex': '306',
  'from': '0x447997d1dc7888a2ffc15826e7595a6f7dcc076b',
  'to': '0xdac17f958d2ee523a2206206994597c13d8

In [ ]:
len(total_rows), len([row for row in total_rows if type(row) == dict])

(486484, 485587)

(501005, 500079)

In [ ]:
# df = pd.DataFrame([row for row in total_rows if type(row) == dict])
# df.head(3)

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations
0,9741106,1585147772,0xaef1b028055d57d27ef3c1c27b52e0081449aaf5b604...,2960,0x4ec40e185edfded6adff507653a0bb03daebb6221406...,65,0x00de4b13153673bcae2616b67bf822500d325fc3,0x01f1e9319982ab828203c0276727cf48065434d3,2500000000000000,21000,10000000000,0,1,0x,,4420950,21000,3543842
1,10395981,1593906601,0x8645d41dfb512084a5f6c60b49caf541086a15cb46a0...,1313684,0x29d4467065cceb46430e358c1f658684845a8d80436f...,40,0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0,0x9b66bcefa1becfe4b988e23cb1017e8f2c402067,85000000000000000,30000,24150000000,0,1,0x,,1147636,21000,2888967
2,10396024,1593907176,0xee5b78df322a3a8a96c9b6e87777ceb3ba375ba5e343...,0,0x58092e4c82e810c3e078f1946ad3a23b4e0559b05d63...,40,0x9b66bcefa1becfe4b988e23cb1017e8f2c402067,0x6b175474e89094c44da98b954eedeac495271d0f,0,44058,23000000000,0,1,0x095ea7b3000000000000000000000000b9a6f2ff1681...,,8117271,44058,2888924


,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations
0,13198991,1631289899,0x19447f4db56074a58ed898d82792b1840cae17cc9f49...,7,0xad8c19b380dbf724d49032badc36a52f85adc3b498d1...,21,0x0cbbc6ae48d9d3e14079aa15fab79a93b9f14796,0x447997d1dc7888a2ffc15826e7595a6f7dcc076b,40000000000000000,21000,201000000000,0,1,0x,,1712459,21000,303446
1,13199002,1631290114,0x9069dd7c39248b635b4d4129a9f747b0fbed1df80b7e...,0,0xceb3d9ed463a983d7472fa197aabebefca533e950aab...,306,0x447997d1dc7888a2ffc15826e7595a6f7dcc076b,0xdac17f958d2ee523a2206206994597c13d831ec7,0,94795,101574817824,0,1,0xa9059cbb000000000000000000000000a5ef5f870771...,,27995552,58397,303435
2,13199011,1631290177,0x2b3732bade815729a849a6fa52b9390bfface387fc2b...,1,0xf447eb0e038a705484ab75593f86bc27767db937ce83...,311,0x447997d1dc7888a2ffc15826e7595a6f7dcc076b,0x0cbbc6ae48d9d3e14079aa15fab79a93b9f14796,30726977495774872,21000,81080397574,0,1,0x,,25089151,21000,303426


In [ ]:
# df.to_csv('savepoint.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32e1581d-cef2-4771-a5bf-9bbfe7a3603c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>